## Сбор данных 

**Импорт библиотек:**

In [7]:
import requests
import numpy as np
import pandas as pd
import yaml

**Считываем токен для доступа к VK API из `.yaml` файла:**

In [8]:
conf = open('config.yaml')

config = yaml.load(conf, yaml.SafeLoader)

access_token = config['access_token']

**Считываем описание признаков пользователей VK из `.yaml` файла для формирования `DataFrame`:**

In [9]:
descr = open('description/description_search.yaml', mode = 'r', encoding = 'utf-8')

description = yaml.load(descr, yaml.SafeLoader)

**Подготовка к API запросу. Используется метод `users.get`:**

In [10]:
url = 'https://api.vk.com/method/users.get?'

params = {
    'v': 5.131,
    'access_token': access_token,
    'fields': 'relation, personal, universities, sex'
}

**Функция, которая 'раскрывает' словарь:**

In [16]:
def flatten(dictionary, key):
        
    if not key in dictionary.keys():
        return

    if type(dictionary[key]) != list:
        for subkey in dictionary[key].keys():
            dictionary[key + '_' + subkey] = dictionary[key][subkey]
        
    elif len(dictionary[key]) != 0:
        for subkey in dictionary[key][0].keys():
            dictionary[key + '_' + subkey] = dictionary[key][0][subkey]
            
    dictionary.pop(key)    
    
    return dictionary

**Функция, которая удаляет ключ из словаря:**

In [12]:
def del_key(dictionary, key):
    
    if not key in dictionary.keys():
        return
    
    dictionary.pop(key)
    
    return dictionary

**Функция, которая получает сведения о пользователях VK путем API запросов и возвращает `DataFrame` с полученными данными.**

**Для быстрого сбора данных используется метод `users.get` с пакетом из 1000 случайных id.**

In [13]:
def get_data(count=1000):
    
    user_str = ''

    user_id_list = np.random.randint(1, 60000000, count)

    for user_id in user_id_list:
        user_str = user_str + str(user_id) + ', '
    
    df = pd.DataFrame(columns=description.keys())
    
    params['user_ids'] = user_str
        
    r = requests.get(url, params = params)
    
    data_dict = r.json()['response']
    
    for user_dict in data_dict:
        
        # Обрабатываем только активных пользователей с непустым полом
        if  (user_dict['first_name'] != 'DELETED') and \
            ('deactivated' not in user_dict.keys()) and \
            (user_dict['is_closed'] == False) and \
            (user_dict['sex'] != 0):
        
            # Раскрываем вложенные словари и удаляем ненужные
            flatten(user_dict, 'personal')
            flatten(user_dict, 'universities')
            del_key(user_dict, 'relation_partner')
            
            df = df.append(user_dict, ignore_index=True)
            
    return df

**Фунция, которая вызывает в цикле предыдущую функцию `get_data()` до тех пор пока не накопит требуемое количество пользователей. Все полученные пользователи сохраняются в `.csv` файл для дальнейшей обработки:**

In [14]:
def get_users_csv(min_count, file_name):
    
    file_name = file_name + '.csv' 
    
    full_df = pd.DataFrame(columns=description.keys())
    
    while len(full_df) < min_count:
    
        df = get_data()
    
        full_df = pd.concat([full_df, df])
        
        full_df.to_csv(file_name, index_label = 'id')
    
    full_df = full_df.drop_duplicates(subset=['id'])
    
    full_df = full_df.set_index('id')
        
    return full_df

**Для того, чтобы при сбое не запускать сбор данных с самого начала, будем извлекать данные пакетами по 25000 пользователей и сохранить их в папку `data`:**

In [ ]:
%%time

file_nums = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for num in file_nums:
    
    get_users_csv(25000, file_name='data/data_'+str(num))